In [692]:
# Import
import pandas as pd
import numpy as np
import matplotlib
import logging
import urllib
import io
import csv
from lxml import etree
from utils.statistics import statistics, bar_chart_simple
from bs4 import BeautifulSoup

# Matplotlib confs
%matplotlib inline
matplotlib.style.use("seaborn")

# Constraints
APPALTI_XSD = "http://dati.anticorruzione.it/schema/datasetAppaltiL190.xsd"
INDICE_APPALTI_XSD = "http://dati.anticorruzione.it/schema/datasetIndiceAppaltiL190.xsd"

TEST_APPALTI="http://www.archiviodistatotrento.beniculturali.it/MW/mediaArchive/Archive/avcp_dataset_2014.xml"
TEST_INDICE="http://www.studiok.it/COMUNI/RONCOBELLO/AVCP/datasetIndiceAppalti.xml"

In [693]:
df = pd.read_csv('data/l190-collected.csv', sep=';')
df.head(2)

,codiceFiscale,dataUltimoTentativoAccessoUrl,esitoUltimoTentativoAccessoUrl,identificativoPEC,ragioneSociale,url,anno,url2,ext
0,80003710789,2015-04-22T18:42:06.137+0000,fallito,opec275.20140526151617.31733.02.1.48@pec.aruba.it,AMMINISTRAZIONE PROVINCIALE DI COSENZA,http://servizi.provincia.cs.it/legge190/2013/i...,2015.0,3/indice_dataset.xml,xml
1,80015990544,2015-04-22T13:25:54.003+0000,fallito,opec275.20150112133830.12508.04.1.6@pec.actali...,DIREZIONE DIDATTICA DI CORCIANO,http://www.circolodidatticocorciano.gov.it/?pa...,2015.0,gov.it/?page_id=4142,altro


In [694]:
df = df.loc[:,('esitoUltimoTentativoAccessoUrl','url', 'anno', 'ext')].copy()
df['anno']= df['anno'].astype('object')

df_suc=df.loc[(df["esitoUltimoTentativoAccessoUrl"] =='successo')].copy()

## Controllo sintattico 
Pur avendo esito della comunicazione positivo, alcuni url non sono formattati correttamente. Di seguito si procede al controllo sintattico riguardante gli spazi e successivamente alla corretta formattazione dell'url.

### Controllo sugli spazi
Gli elementi contenete spazi nell'attributo url sono:

In [695]:
# Controlla url con spazi
df_suc_with_blank_in_url= df_suc[df_suc['url'].str.contains(" ", na=np.nan)==True].copy()
df_suc_with_blank_in_url.shape[0]

2298

In [696]:
# Applica eliminazione spazi
df_suc['url']=df_suc['url'].apply(lambda x: x.strip('\n').strip(' ').replace(" ", "").replace("\t", ""))

# Controlla url con spazi
df_suc_with_blank_in_url= df_suc[df_suc['url'].str.contains(" ", na=np.nan)==True].copy()
df_suc_with_blank_in_url.shape[0]

0

### Controllo sintassi url nella forma corretta forma
Gli elmenti url che on conminciano con http:// or https:// sono:

In [697]:
df_suc_with_wrong_start=df_suc[df_suc['url'].str.match('http|HTTP(s|S)?\:\/\/*')==False].copy()
print ("Url non correttamente formattati:", df_suc_with_wrong_start.shape[0])

Url non correttamente formattati: 583


In [698]:
# Trova tutti quelli elementi che non soddisfano la condizione http|HTTP(s|S)?\:\/\/* 
df_suc['url_wrong']=df_suc['url'].str.match('http|HTTP(s|S)?\:\/\/*')==False

# Applica trasformazione a url cosi da sistemarl
df_suc['url']=df_suc.apply(lambda x: "http://"+x.url if x.url_wrong else x.url, axis=1)

#Cancella colonna url_wrong
df_suc = df_suc.drop('url_wrong', 1)

In [699]:
# Ricontrollo
df_suc_with_wrong_start=df_suc[df_suc['url'].str.match('http|HTTP(s|S)?\:\/\/*')==False].copy()
print ("Url non correttamente formattati:", df_suc_with_wrong_start.shape[0])

Url non correttamente formattati: 0


## Estratore xml

In [700]:
def get_node(node, field):
    try:
        result = node.find(field).text
    except AttributeError:
        result = np.nan

    return result

def serialize_field(node, field):
    result = ','.join(y.text for y in node.find_all(field))
    result = np.nan if len(result)==0 else result
    return result

def xml_to_csv(doc):
    soup = BeautifulSoup(etree.tostring(doc), "lxml")
    lotti = soup.find_all("lotto")
            
    ente = soup.metadata.entepubblicatore.text
    data_ultimo_aggiornamento = soup.metadata.dataultimoaggiornamentodataset.text
            
    path_to_csv = "data/appalti/%s_%s.csv" % (ente.replace(" ", "_"), data_ultimo_aggiornamento.replace(" ", "_"))
    csv_file = open(path_to_csv, 'w')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(np.array([
        'cig','codicefiscale_prop', 'denominazione', 'oggetto','scelta_contraente',
        'partecipanti','aggiudicatari','importo_aggiudicazione','data_inizio','data_fine',
        'importo_somme_liquidate']))

    for i in range (0, len(lotti)):
        x = lotti[i]
        csv_writer.writerow(np.array([
            get_node(x,"cig"),
            get_node(x.find("strutturaproponente"), "codicefiscaleprop"),
            get_node(x.find("strutturaproponente"), "denominazione"),
            get_node(x,"oggetto"),
            get_node(x,"sceltacontraente"),
            serialize_field(x.partecipanti, "codicefiscale"),
            serialize_field(x.partecipanti, "aggiudicatari"),
            get_node(x,"importoaggiudicazione"),
            get_node(x.tempicompletamento, "datainizio"),
            get_node(x.tempicompletamento, "dataultimazione"),
            get_node(x, "importosommeliquidate")  
        ]))
        
    csv_file.close()

In [701]:
# Ricavare lo schema degli appalti
appalti_xmlschema_doc = etree.parse(APPALTI_XSD)
appalti_xmlschema = etree.XMLSchema(appalti_xmlschema_doc)
# Ricavare lo schema degli indice appalti
indice_xmlschema_doc = etree.parse(INDICE_APPALTI_XSD)
indice_xmlschema = etree.XMLSchema(indice_xmlschema_doc)


# UnComment for test
df_suc.drop(df_suc.loc[50:].index, inplace=True);

In [702]:
logger = logging.getLogger('xml_collector_flow')
logger.setLevel(logging.DEBUG)

# create a file handler
%rm log/xml_collector_flow.log
handler = logging.FileHandler('log/xml_collector_flow.log')
handler.setLevel(logging.DEBUG)

# create a logging format
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# add the handlers to the logger
logger.addHandler(handler)

# url = np.array url opportunatamente parametrizzato in modo da cominciare da un dato elemento
xml_indice = np.zeros(0, dtype=int)
xml_schema_errato = np.zeros(0, dtype=int)
xml_errati = np.zeros(0, dtype=int)
counter_so_far = 0
start = 0
logger.info("=== Start ===")

# Per ogni u in url:
for u in df_suc.loc[start:, 'url'].values:
    # logga il numero elemento iniziato
    msg = "Comincia elemento %d: %s" %(counter_so_far, u)
    logger.info(msg)
    
    # Prova a scaricare l'xml se va in errore 
    try:
        
        #logger.debug(type(xml))
        doc = etree.parse(u)
        # se valida_appalti(u):
        if appalti_xmlschema.validate(doc):
            # parserizza e scarica
            xml_to_csv(doc)
        # se valida_indice(u):
        elif indice_xmlschema.validate(doc):
            # non parserizzare
            xml_indice=np.append(xml_indice,counter_so_far)
            # scrivi elemento in xml_indici(np.array)
        # altrimenti
        else:
            # scrivi elemento in xml_non_validi
            xml_schema_errato=np.append(xml_schema_errato,counter_so_far)
    except (urllib.error.URLError, urllib.error.HTTPError, OSError, etree.XMLSyntaxError) as e:
        # scrivi elemento in xml_errori
        xml_errati= np.append(xml_errati,counter_so_far)
    
    counter_so_far +=1
    
    

INFO:xml_collector_flow:=== Start ===
INFO:xml_collector_flow:Comincia elemento 0: http://omnicomprensivocerretodispoleto.it/avcp/trasparenza/11/2014.xml
INFO:xml_collector_flow:Comincia elemento 1: http://www.consorzioinviolatellasalaria.it/AVCP/2014/dataset2.xml
INFO:xml_collector_flow:Comincia elemento 2: http://www.icsanmartinovc.gov.it/amministrazione-trasparente/bandi-di-concorso/informazioni-di-cui-alla-deliberazione-avcp-n26-del-22052013/item/download/211
INFO:xml_collector_flow:Comincia elemento 3: http://www.comune.buccinasco.mi.it/openweb/web/benefici/appalti/pubblicazione_legge190_2014.xml
INFO:xml_collector_flow:Comincia elemento 4: http://www.studiok.it/COMUNI/RONCOBELLO/AVCP/datasetIndiceAppalti.xml
INFO:xml_collector_flow:Comincia elemento 5: http://www.archiviodistatotrento.beniculturali.it/MW/mediaArchive/Archive/avcp_dataset_2014.xml
INFO:xml_collector_flow:Comincia elemento 6: http://www.archeologicamolise.beniculturali.it/getFile.php?id=1311
INFO:xml_collector_flow

In [708]:
len(xml_errati)+len(xml_indice)+len(xml_schema_errato)

13